<a href="https://colab.research.google.com/github/rjlobosco/elem_fini_2022/blob/main/Mecflu_Dataframes_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTAÇÕES E INSTALAÇÕES

!pip install openpyxl
import math
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# CONSTANTES, INPUTS E FUNÇÕES


# Constantes
PA = 101.325  # Pressão atmosférica ao nível do mar em kPa
k = 1.15e-4   # Constante em s²/m²
g = 9.81      # Aceleração gravitacional em m/s²

# Função para calcular a altura a partir da pressão
def calcular_altura(PA, k, P_h, g):
  # P_h: Pressão a uma altura h (em KPa)
  altura = -math.log(P_h / PA) / (k * g)
  return altura

# Função para converter horário de Brasília para UTC
def brasilia_to_utc(hour_brasilia):
  brasilia_time = datetime.strptime(hour_brasilia, '%H:%M')
  utc_time = brasilia_time + timedelta(hours=3)
  return utc_time.strftime('%H:%M')


# Função para converter temperatura de Kelvin para Celsius
def kelvin_to_celsius(kelvin):
  return kelvin - 273.15

# Função para verificar se a data é válida
def data_valida(dia, mes, ano):
  try:
      datetime(ano, mes, dia)
      return True
  except ValueError:
      return False


def estacao_do_ano(dia, mes):
  """
  Retorna a estação do ano no hemisfério sul com base na data fornecida.

  Parâmetros:
  dia (int): O dia do mês.
  mes (int): O mês (1-12).

  Retorna:
  str: A estação correspondente ('Verão', 'Outono', 'Inverno', 'Primavera').
  """

  # Verão: 21 de Dezembro a 20 de Março
  if (mes == 12 and dia >= 21) or (1 <= mes <= 2) or (mes == 3 and dia <= 20):
      return 'Verão'
  # Outono: 21 de Março a 20 de Junho
  elif (mes == 3 and dia >= 21) or (4 <= mes <= 5) or (mes == 6 and dia <= 20):
      return 'Outono'
  # Inverno: 21 de Junho a 22 de Setembro
  elif (mes == 6 and dia >= 21) or (7 <= mes <= 8) or (mes == 9 and dia <= 22):
      return 'Inverno'
  # Primavera: 23 de Setembro a 20 de Dezembro
  elif (mes == 9 and dia >= 23) or (10 <= mes <= 11) or (mes == 12 and dia <= 20):
      return 'Primavera'

# Níveis de pressão em KPa
pressure_levels = [70.0, 75.0, 77.5, 80.0, 82.5, 85.0, 87.5, 90.0, 92.5, 95.0, 97.5, 100.0]

# Horas
horas = ['03:00', '09:00', '15:00', '21:00']

# Dias, meses e anos
anos = [2023]
meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
dias = [d for d in range(1, 32)]


# Coordenadas das plataformas
platforms = {
    'NAMORADO 2 (PNA-2)': (-22.45073, -40.41175),
    'PETROBRAS 26 (P-26)': (-22.4684, -40.02869),
    'PETROBRAS 32 (P-32)': (-22.2051, -40.1431),
    'PETROBRAS 37 (P-37)': (-22.4868, -40.09779),
    'PETROBRAS IX': (-22.57358, -40.82192),
    'PETROBRAS XIX': (-22.3927, -40.05438),
    'PETROBRAS XXXIII': (-22.37, -40.0267),
    'VERMELHO 1 (PVM-1)': (-22.16065, -40.27872),
    'VERMELHO 2 (PVM-2)': (-22.17535, -40.29147),
}

season_names = {
        'DJF': 'Verão',
        'MAM': 'Outono',
        'JJA': 'Inverno',
        'SON': 'Primavera'
        }


In [ ]:
 # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
nc_filename = 'Era5_2023_CAMPOS.nc'

# Carrega o arquivo NetCDF usando xarray
ds = xr.open_dataset(nc_filename, engine='h5netcdf')

# Renomeia a coordenada 'valid_time' para 'time'
ds = ds.rename({'valid_time': 'time'})

In [ ]:
# Cria um DataFrame vazio para armazenar os resultados combinados
combinado_df = pd.DataFrame()

# Nomeia o índice do DataFrame
combinado_df.index.name = 'Índice'

# Loop sobre cada ano
for ano in anos:

  # Cria um DataFrame vazio para armazenar os resultados combinados dos meses
  mes_df = pd.DataFrame()

  # Loop sobre cada mês
  for mes in meses:

    # Cria um DataFrame vazio para armazenar os resultados combinados dos dias
    dia_df = pd.DataFrame()

    # Loop sobre cada dia
    for dia in dias:

      # Verifica se a data existe
      if not data_valida(dia, mes, ano):
        continue  # Pula esta iteração se o dia não for válido

      # Cria um DataFrame vazio para armazenar os resultados combinados dos níveis de pressão
      level_df = pd.DataFrame()

      # Loop sobre cada nível de pressão
      for level in pressure_levels:

        # Cria um DataFrame vazio para armazenar os resultados combinados das horas
        hora_df = pd.DataFrame()

        # Loop sobre cada hora
        for hora in horas:

          #Função para obter a hora UTC (Tempo Universal Coordenado) a partir da hora de Brasília
          hora_utc = brasilia_to_utc(hora)

          # Filtros
          filtra_hora = ds.time.dt.strftime('%H:%M') == hora_utc
          filtra_level = ds['pressure_level'] == level*10
          filtra_dia = ds.time.dt.day == dia
          filtra_mes = ds.time.dt.month == mes
          filtra_ano = ds.time.dt.year == ano

          # Aplicação dos filtros com where
          ds_f = ds.where(filtra_hora & filtra_level & filtra_dia & filtra_mes & filtra_ano, drop=True)

          # DataArray das velocidades
          u = ds_f['u']
          v = ds_f['v']

          # Calcula a velocidade do vento (wspd)
          wspd = (u**2 + v**2)**0.5

          # DataArray das temperaturas
          temp = ds_f['t']

          # Cria um DataFrame vazio para armazenar os resultados combinados das plataformas
          unico_df = pd.DataFrame()

           # Loop sobre cada plataforma
          for platform, (lat, lon) in platforms.items():

            # Interpola o valor das componentes da velocidade do vento
            vento_u = u.interp(latitude=lat, longitude=lon, method='nearest').values.item()
            vento_v = v.interp(latitude=lat, longitude=lon, method='nearest').values.item()
            # Interpola o valor da média da velocidade do vento
            vento_resultante = wspd.interp(latitude=lat, longitude=lon, method='nearest').values.item()

            # Interpola o valor da média da temperatura em Kelvin
            temp_kelvin = temp.interp(latitude=lat, longitude=lon, method='nearest').values.item()
            # Converte a temperatura para Celsius
            temp_celsius = kelvin_to_celsius(temp_kelvin)

            # Chama a função que determina a estação do ano a partir da data
            estacao = estacao_do_ano(dia, mes)

            # Adiciona uma linha com cada coluna para cada plataforma
            unico_df = pd.concat([unico_df, pd.DataFrame({
                      'Plataforma': platform,
                      'Velocidade_Vento_u_m/s': vento_u,
                      'Velocidade_Vento_v_m/s': vento_v,
                      'Velocidade_Vento_media_m/s': vento_resultante,
                      'Temperatura_K': temp_kelvin,
                      'Temperatura_C': temp_celsius,
                      'Dia': dia,
                      'Mês': mes,
                      'Ano': ano,
                      'Estação_do_Ano': estacao
                  }, index=[0])], ignore_index=True)


          #Adiciona as colunas de hora
          unico_df['Horário_Brasília'] = hora
          unico_df['Horário_UTC'] = hora_utc

          # Adiciona o DataFrame unico atual ao DataFrame da hora atual
          hora_df = pd.concat([hora_df, unico_df], ignore_index=True)

        # Adiciona a coluna de pressão
        hora_df['Nível_de_Pressão_hPa'] = int(level*10)

        # Calcula a altura correspondente ao nível de pressão atual
        altura = calcular_altura(PA, k, level, g)
        # Adiciona a coluna de altura
        hora_df['Altitude_m'] = altura

        # Adiciona o DataFrame da hora atual ao DataFrame de nível da pressão atual
        level_df = pd.concat([level_df, hora_df], ignore_index=True)

      # Adiciona a coluna de dia
      level_df['Dia'] = dia
      # Adiciona o DataFrame do nível de pressão atual ao DataFrame do dia atual
      dia_df = pd.concat([dia_df, level_df], ignore_index=True)

    # Adiciona a coluna de mês
    dia_df['Mês'] = mes
    # Adiciona o DataFrame do dia atual ao DataFrame do mês atual
    mes_df = pd.concat([mes_df, dia_df], ignore_index=True)

  # Adiciona a coluna de ano
  mes_df['Ano'] = ano
  # Adiciona o DataFrame do nível do mês atual ao DataFrame combinado
  combinado_df = pd.concat([combinado_df, mes_df], ignore_index=True)


  # Fecha o arquivo NetCDF
  ds.close()



In [ ]:
# Criar a coluna 'Data' a partir das colunas 'Ano', 'Mês' e 'Dia'
combinado_df['Data'] = pd.to_datetime(combinado_df[['Ano', 'Mês', 'Dia']].rename(columns={'Ano':'year', 'Mês':'month', 'Dia':'day'}), errors='coerce') # Rename columns to match expected names

# Remover as colunas 'Ano', 'Mês' e 'Dia'
combinado_df = combinado_df.drop(columns=['Ano', 'Mês', 'Dia'])

In [ ]:
# Ajusta a ordem das colunas
combinado_df = combinado_df[['Plataforma', 'Nível_de_Pressão_hPa', 'Altitude_m', 'Estação_do_Ano', 'Data', 'Horário_Brasília', 'Horário_UTC',
                           'Velocidade_Vento_u_m/s', 'Velocidade_Vento_v_m/s', 'Velocidade_Vento_media_m/s', 'Temperatura_K', 'Temperatura_C']]



In [ ]:
# Salvar o DataFrame combinado como um arquivo CSV
csv_filename = f'2023_DataFrame.csv'
combinado_df.to_csv(csv_filename, encoding='utf-8', index=True)

#Salvar o DataFrame combinado como um arquivo Excel
excel_filename = f'2023_DataFrame.xlsx'
combinado_df.to_excel(excel_filename, index=True)